In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy as np
import pandas as pd
from glob import glob
import pickle

"""Change to the data folder"""
new_path = "./new_train/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
sub = pd.read_csv('sample_submission.csv')

In [5]:
sub.head()

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
from pathlib import Path

pathlist = Path('./new_train/new_train').glob('**/*.pkl')
for path in pathlist:
     # because path is object not string
     path_in_str = str(path)
     print(path_in_str)

In [45]:
for path in pathlist:
     path_in_str = str(path)
     print(path_in_str)

<generator object Path.glob at 0x7f59a067c7d0>

In [9]:
infile = open("./new_train/new_train/0.pkl",'rb')
new_dict = pickle.load(infile)
infile.close()

In [29]:
new_dict.keys()

dict_keys(['city', 'lane', 'lane_norm', 'scene_idx', 'agent_id', 'car_mask', 'p_in', 'v_in', 'p_out', 'v_out', 'track_id'])

In [41]:
object_x = new_dict['p_in'][new_dict['car_mask'].reshape([-1]).astype(bool),:,0]

In [53]:
object_x.flatten()

array([3277.29638672, 3277.29614258, 3277.29614258, 3277.29614258,
       3277.29589844, 3277.29589844, 3277.29589844, 3277.29614258,
       3277.29638672, 3277.29614258, 3277.29614258, 3277.29638672,
       3277.29638672, 3277.29663086, 3277.29638672, 3277.29638672,
       3277.29638672, 3277.29638672, 3277.29638672, 3277.13671875,
       3277.09350586, 3277.22485352, 3277.17895508, 3277.08569336,
       3277.10571289, 3277.09033203, 3277.10595703, 3277.01708984,
       3277.1394043 , 3277.14135742, 3277.19213867, 3277.10546875,
       3277.10205078, 3277.18115234, 3277.13842773, 3277.07397461,
       3277.16162109, 3277.17016602, 3232.28955078, 3232.33007812,
       3232.28295898, 3232.26635742, 3232.29907227, 3232.30688477,
       3232.27587891, 3232.23339844, 3232.27490234, 3232.29516602,
       3232.25390625, 3232.28076172, 3232.28735352, 3232.29199219,
       3232.31420898, 3232.29467773, 3232.28857422, 3232.35107422,
       3232.2590332 , 3275.30639648, 3275.29418945, 3275.31469

In [20]:
new_dict['car_mask'].reshape([-1]).astype(bool)

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [27]:
new_dict['track_id'].reshape([-1]).shape

(1800,)

### Create a dataset class 

In [6]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

In [54]:
x = []
y = []
v = []
for data in val_dataset:
    x += data['p_in'][data['car_mask'].reshape([-1]).astype(bool),:,0].flatten()
    y += data['p_in'][data['car_mask'].reshape([-1]).astype(bool),:,1].flatten()

ValueError: operands could not be broadcast together with shapes (0,) (114,) 

### Create a loader to enable batch processing

In [7]:
batch_sz = 1

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [np.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [8]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])


In [18]:
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
for i_batch, sample_batch in enumerate(val_loader):
    
    inp, out = sample_batch
    """TODO:
      Deep learning model
      training routine
    """
    print(inp.shape)
    print(inp[:,:,:,0])
    #show_sample_batch(sample_batch, agent_id)
    break

torch.Size([1, 60, 19, 4])
tensor([[[3277, 3277, 3277,  ..., 3277, 3277, 3277],
         [3277, 3277, 3277,  ..., 3277, 3277, 3277],
         [3232, 3232, 3232,  ..., 3232, 3232, 3232],
         ...,
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]]])


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


flatten = nn.Flatten()
lstm = nn.LSTM((60,19,4))


model = NeuralNetwork().to(device)
print(model)

In [35]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [36]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [37]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(val_loader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Expected object of scalar type Float but got scalar type Long for argument #4 'mat1'